<a href="https://colab.research.google.com/github/MostaryKhatun/AdvanceCriptography/blob/main/PRNG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# XorWeyMixPRNG: Custom pseudo-random number generator
# Safe to run in Google Colab

import time
import struct

class XorWeyMixPRNG:
    """
    XorWeyMixPRNG: A lightweight 64-bit pseudo-random number generator.
    Combines a xorshift update, a Weyl sequence increment, and a nonlinear mixing step.
    Non-cryptographic but suitable for simulation or ML sampling.
    """

    def __init__(self, seed=None):
        if seed is None:
            seed = int(time.time_ns()) & 0xFFFFFFFFFFFFFFFF
        self.state = seed & 0xFFFFFFFFFFFFFFFF
        self.weyl = 0x61C8864680B583EB  # Weyl constant (~ golden ratio * 2^64)
        self.add = 0xA3EC647659359ACD  # arbitrary odd constant
        self._mix_const = 0xD2B74407B1CE6E93

    def seed(self, seed_val):
        self.state = seed_val & 0xFFFFFFFFFFFFFFFF

    def _rotl(self, x, k):
        return ((x << k) | (x >> (64 - k))) & 0xFFFFFFFFFFFFFFFF

    def _mix64(self, z):
        z = (z ^ (z >> 30)) * 0xBF58476D1CE4E5B9 & 0xFFFFFFFFFFFFFFFF
        z = (z ^ (z >> 27)) * 0x94D049BB133111EB & 0xFFFFFFFFFFFFFFFF
        return z ^ (z >> 31)

    def rand_uint64(self):
        """Return next 64-bit unsigned integer."""
        self.state = (self.state ^ (self.state >> 12)) & 0xFFFFFFFFFFFFFFFF
        self.state = (self.state ^ (self.state << 25)) & 0xFFFFFFFFFFFFFFFF
        self.state = (self.state ^ (self.state >> 27)) & 0xFFFFFFFFFFFFFFFF
        self.state = (self.state * self.add + self.weyl) & 0xFFFFFFFFFFFFFFFF
        mixed = self._mix64(self.state + self._mix_const)
        return mixed

    def rand(self):
        """Return float in [0, 1)."""
        return self.rand_uint64() / 2**64

    def randint(self, a, b):
        """Return random integer between a and b inclusive."""
        return a + self.rand_uint64() % (b - a + 1)

    def choice(self, seq):
        """Return random element from sequence."""
        if not seq:
            raise ValueError("Empty sequence")
        return seq[self.randint(0, len(seq) - 1)]

    def shuffle(self, seq):
        """Shuffle sequence in-place."""
        n = len(seq)
        for i in range(n - 1, 0, -1):
            j = self.randint(0, i)
            seq[i], seq[j] = seq[j], seq[i]


# ======== DEMO ========
if __name__ == "__main__":
    rng = XorWeyMixPRNG(123456789)

    print("Sample 64-bit integers:")
    for _ in range(5):
        print(hex(rng.rand_uint64()))

    print("\nSample floats [0,1):")
    for _ in range(5):
        print(rng.rand())

    # Basic statistical sanity check
    N = 200_000
    vals = [rng.rand() for _ in range(N)]
    mean_val = sum(vals) / N
    bits = [int(v * (1 << 64)) & 1 for v in vals]
    low_ones = sum(bits)
    low_zeros = N - low_ones

    print(f"\nmean ~ {mean_val:.6f}")
    print(f"low-bit ones: {low_ones}, zeros: {low_zeros}")

    # Save code to file for reusability
    code_text = """{}""".format(open(__file__).read() if '__file__' in globals() else '')
    with open("xorweymix_prng.py", "w") as f:
        f.write(code_text)

    print("\nFile 'xorweymix_prng.py' saved in current Colab directory.")


Sample 64-bit integers:
0xa70ad1228f9c1f84
0x700e18dfb36dae4f
0xa1d39b474743bb30
0xf20864ed0fcca423
0xefba34d0d07c8d21

Sample floats [0,1):
0.7656563662884213
0.38514277827129834
0.5217711492823698
0.14690188957257155
0.351261623924686

mean ~ 0.499516
low-bit ones: 64, zeros: 199936

File 'xorweymix_prng.py' saved in current Colab directory.
